<a href="https://colab.research.google.com/github/JakobUniver/algorithmics_3D_maze/blob/main/evolutionary_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic algorithm
This algorithm goal is to find the best direction to thow a ball to hit the target.

In [63]:
# imports
import numpy as np
import time

In [87]:
np.array([1,2,3]).shape
np.zeros((3,1)).shape

(3, 1)

In [6]:
a = np.array([[[-4,5]],[[12,23]], [[1,1]]])
print(a)
print(a[:,0])

[[[-4  5]]

 [[12 23]]

 [[ 1  1]]]
[[-4  5]
 [12 23]
 [ 1  1]]


In [90]:
np.concatenate([[1,2],[4,5]])

array([1, 2, 4, 5])

In [95]:
class Evolution:
  xz_range:tuple = 360
  yz_range:tuple = 360

  def __init__(self,population_size):
    self.population_size = population_size
    self.population = self.init_population()
  
  def fitness(self,instance: np.array)->float:
    # TODO
    return abs(instance.sum())

  def init_population(self)->list:
    population = np.random.random((self.population_size,2))
    # Next we do because we do not want that the range endpoints are included
    population[:,0] = population[:,0]*(self.xz_range/2)
    population[:,1] = population[:,1]*(self.yz_range/2)
    population[:,0] = population[:,0]*np.random.choice([-1,1],self.population_size)
    population[:,1] = population[:,1]*np.random.choice([-1,1],self.population_size)
    population_valuation = np.array([self.fitness(chromosome) for chromosome in population])
    return population_valuation, population

  def mutation(self,chromosome:np.array, nr_genes:int=6)->tuple:
    # https://en.wikipedia.org/wiki/Mutation_(genetic_algorithm)
    xz_dif = np.random.normal(0,self.xz_range/nr_genes)
    yz_dif = np.random.normal(0,self.yz_range/nr_genes)

    mutant = np.array([chromosome[0]+xz_dif,chromosome[1]+yz_dif])
    
    if abs(mutant[0])>self.xz_range/2:
      mutant[0] = (self.xz_range - abs(mutant[0])) *(1 if mutant[0]<0 else -1)
    if abs(mutant[1])>self.yz_range/2:
      mutant[1] = (self.xz_range - abs(mutant[1])) *(1 if mutant[1]<0 else -1)
      
    return self.fitness(mutant),mutant

  def crossover(self,parents:np.array)->list:
    parent1,parent2 = parents

    child1_val,child1 = self.mutation([parent1[0],parent2[1]])
    child2_val,child2 = self.mutation([parent2[0],parent1[1]])

    return [child1_val,child2_val],[child1,child2]

  def generate_pairs(self,parents:tuple,size:int)-> list:
    parent_valuations,parent_individuals = parents
    valuations = 1/parent_valuations # Make smaller values bigger and bigger values smaller
    parents_probabilites = valuations / valuations.sum()
    chosen_parents_i = np.random.choice(range(len(parents_probabilites)),size*2,p=parents_probabilites)# ,replace=False
    chosen_parents = parent_individuals[chosen_parents_i]
    pairs = np.array(list(zip(chosen_parents[::2],chosen_parents[1::2])))
    return pairs
  
  def generate_offspring(self)->tuple:
    parents = self.generate_pairs(self.population,self.population_size//4)
    children_val= []
    children= []
    for pair in parents:
      children_val_i, children_i=self.crossover(pair)
      children_val+=children_val_i
      children+=children_i

    return children_val,children_i

  def choose_new_generation(self, population:tuple)->tuple:
    valuations = 1/population[0]# Make smaller values bigger and bigger values smaller
    chromo_probabilites = valuations / valuations.sum()
    chosen_chromos_i = np.random.choice(range(len(chromo_probabilites)),self.population_size,p=chromo_probabilites,replace=False)
    new_population = (population[0][chosen_chromos_i],population[1][chosen_chromos_i])
    return new_population

    
  def step(self)->list:
    children = []
    
    # Generate new offspring
    children = self.generate_offspring()
    new_population_val = np.concatenate([self.population[0],children[0]])
    new_population_ins = np.concatenate([self.population[1],children[1]])
    new_population = (new_population_val,new_population_ins)

    # Let the natural selection to do its job
    self.population = self.choose_new_generation(new_population)

    return self.population

  def step_until(self,max_iters:int=1000, max_non_increasing_iters:int=100, max_time_s:int=10):
    iters = 0
    non_increasing_iters = 0
    best_val = float('inf')
    progress_vals = []
    progress_ins = []
    start_time = time.perf_counter()

    while True:
      # Check for reasons to stop evolution
      if iters == max_iters:
        print('Stopped because maximum iterations achieved.')
        break
      if non_increasing_iters == max_non_increasing_iters:
        print('Stopped because maximum non increasing iterations achieved.')
        break
      if time.perf_counter() - start_time >= max_time_s:
        print('Stopped because maximum time limit achieved.')
        break
      
      # Produce the next generation
      generation_i_vals,generation_i_ins = self.step()

      # Find the best individual
      best_i_idx = generation_i_vals.argmin()
      best_i_val = generation_i_vals[best_i_idx]
      best_i_in = generation_i_ins[best_i_idx]
      progress_vals.append(best_i_val)
      progress_ins.append(best_i_in)
 
      # Increase counters
      iters +=1
      non_increasing_iters +=1

      # Check if there is an improvement
      if best_i_val < best_val:
        best_val=best_i_val
        non_increasing_iters = 0
    
    return np.array(progress_vals),np.array(progress_ins)
      
obj = Evolution(5)
print(f'Population size: {obj.population_size}')
print(f'Population:\n{obj.population}')
print()
print(obj.mutation([0,0]))
print()
print(obj.crossover([[-45,45],[0,-30]]))
print()
print(obj.generate_pairs((
    np.array([10,    25,     23,      15,      100]),
    np.array([[-4,5],[12,23],[-15,25],[75,-55],[-36,23]])),
    3))

print()
print(obj.generate_offspring())
print()
obj.step()
print()
obj.step_until()
1

Population size: 5
Population:
(array([134.59481737,  61.13662542, 180.87659318,  59.49886058,
        98.78278656]), array([[-120.25651163,  -14.33830574],
       [-102.12260374,  163.25922916],
       [-123.49293723,  -57.38365595],
       [  14.55877693,   44.94008365],
       [ -20.69572231,  -78.08706425]]))

(117.88795599121556, array([-54.56546076, -63.32249523]))

([17.74133265320537, 1.4646927599393678], [array([-31.81647446,  14.07514181]), array([-83.32817384,  84.7928666 ])])

[[[-15  25]
  [ -4   5]]

 [[ 75 -55]
  [ -4   5]]

 [[ 75 -55]
  [ 75 -55]]]

([141.1427792766939, 148.408181304356], [array([ -13.67423307, -127.46854621]), array([-21.02419893, 169.43238024])])


Stopped because maximum non increasing iterations achieved.


1